# Lab 3 - oczyszczanie danych

Dane w surowej formie nie zawsze nadają się do trenowania modeli predykcyjnych. Jedną z najczęstszych przyczyn zanieczyszczenia danych są brakujące wartości w poszczególnych atrybutach. Istnieją trzy popularne rozwiązania ,,walki" z wartościami brakującymi:
- usunięcie wierszy z wybrakowanymi wartościami,
- usunięcie całego atrybutu z wartościami wybrakowanymi,
- uzupełnienie wybranych wartości wybraną strategią.

Dobór rozwiązania w dużym stopniu zależy od charakteru zbioru danych oraz wartości wybrakowanych. W najbardziej idealnym scenariuszu należy sprawdzić efekt zapewniany przez wszystkie poznane rozwiązania, lecz w przypadku ograniczonych zasobów obliczeniowych może nie zawsze być to osiągalne. W praktyce można spotkać się z intyicyjnymi podejściami, które polegają na usuwaniu wierszy z brakującymi wartościami w przypadku ich niewielkiej liczebności, np. do ok. 0.1% oryginalnego zbioru danych. Podejście na usunięciu całego atrybutu sprawdza się gdy znaczna częśc wartości atrybutu jest wybrakowana. W pozostałych przypadkach najlepiej powinno sprawdzić się uzupełnianie wartości według obranej strategii. Wśród najbardziej typowych strategii uzupełniania wartości wybrakowanych warto wymienić następujące:
- średnia arytmetyczna lub mediana (dla wartości rzeczywistych),
- dominanta (dla danych kategorialnych).

Wśród popularnych podejść można znaleźć także związane z wytrenowanymi predyktorami:
- kNN: dopasowuje brakującą wartość na podstawie najbliższych sąsiadów pozostałych uzupełnionych wartości,
- regresor: dopasowuje brakującą wartość na podstawie modelu regresyjnego wytrenowanego na podstawie uzupełnionych wartości.

Wśród pozostałych metod warto zwrócić także uwagę na kontekst atrybutu, w którym występuje brakująca wartość. Przykładowo, jeżeli wybrakowana wartość dotyczy atrybutu poziomu leukocytów we krwi, warto sprawdzić jakie są normy, a następnie wylosować na podstawie wybranego rozkładu prawdopodobieństwa (np. gaussowskiego) wartość w przedziale stanowiącym normę populacji.

## Oczyszczanie danych na podstawie funkcjonalności biblioteki pandas

In [ ]:
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing(as_frame=True)['frame']

In [ ]:
data

Metoda *isnull* wywołana na obiekcie klasy Series zwróci serię wartości logicznych odpowiadających temu czy dana wartośc w kolumnie jest wybrakowana.

In [ ]:
data['MedInc'].isnull()

Za pomocą metody *any* wywołanej na powstałej w ten sposób ramce można sprawdzić czy występuje tam przynajmniej jedna wartość prawdziwa. Metoda all umożliwia sprawdzenie czy w ramce występują tylko i wyłącznie wartości prawdziwe.

In [ ]:
data['MedInc'].isnull().any()

Za pomocą odwołania się do osi kolumn (parametr *axis=0* w metodzie *any*) można z łatwością sprawdzić w których kolumnach występują wartości wybrakowane.

In [ ]:
data.isnull().any(axis=0)

In [ ]:
# przykladowy kod usuwajacy kilka wartosci metoda chybil-trafil
from random import randint

# minimalny i maksymalny odsetek komorek do usuniecia wartosci
min_percent, max_percent = 0.001, 0.003

# wyznacza pseudolosowo od 0.1 do 0.3% komórek z ramki danych
cells_to_remove = randint(int(data.size * min_percent), int(data.size * max_percent))

# pseudolosowy wybor indeksow wierszy i kolumn
for _ in range(cells_to_remove):
  row_idx = randint(0, data.shape[0] - 1)  # pseudolosowy indeks wiersza
  col_idx = randint(0, data.shape[1] - 1)  # pseudolosowy indeks kolumny

  # usuniecie pseudolosowo wskazanej komorki
  data.iat[row_idx, col_idx] = None

In [ ]:
data.isnull().any(axis=0)

Jeżeli w kolumnie występują wartości wybrakowane, można je uzupełnić wskazaną wartością.

In [ ]:
data['MedInc'].fillna(0)

Aby umieścić wartości wybrakowane w oryginalnej ramce danych należy użyć metody *fillna* na obiekcie klasy DataFrame przekazując jako parametr słownik mapujący nazwy kolumn na wartości, którymi mają zostać zastąpione wartości wybrakowane.

In [ ]:
data.fillna({
    'Longitude': 0,
    'Latitude': 100,
})

Biblioteka pandas dostarcza metod wyznaczających podstawowe (i bardziej zaawansowane) statystyki. Na szczególną uwagę zasługują metody *mean()* i *median()*, które zwracają odpowiednio średnią arytmetyczną i medianę wartości ze wskazanego atrybutu.

In [ ]:
data['MedHouseVal'].mean()

In [ ]:
data['MedHouseVal'].median()

W przypadku wyznaczania dominanty zastosowanie znajduje metoda *mode()*. Należy jednak pamiętać, że dominantą nie zawsze musi być tylko jedna wartość.

In [ ]:
data['Population'].mode().iloc[0]

## Oczyszczanie danych z biblioteką Scikit-learn

Oprócz biblioteki *pandas*, biblioteka *Scikit-learn* zawiera także obszerny zestaw narzędzi do pracy z oczyszczaniem danych, w szczególności z uzupełnianiem danych wybrakowanych. Przeznaczona do tego celu klasa *SimpleImputer* przyjmuje w inicjalizatorze parametr *strategy*, w którym należy wskazać metodę uzupełniania brakujących wartości:
- mean,
- median
- most_frequent,
- constant.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

Przed właściwym uzupełnieniem wartości należy najpierw wybrać zestaw pasujących atrybutów, dla których zostanie zastosowana wybrana strategia.

In [ ]:
num_attributes = data.select_dtypes(include=['number'])

In [ ]:
num_attributes

In [ ]:
num_attributes.isnull().any(axis=0)

Wywołanie metody *fit* na utworzonym obiekcie pozwoli na automatyczne wyznaczenie wartości do uzupełnienia w każdym z atrybutów.

In [ ]:
imputer.fit(num_attributes)

W atrybucie *statistics_* zawarte są wyznaczone wartości do zastąpenia brakujących według obranej strategii.

In [ ]:
imputer.statistics_

Do zastąpienia brakujących wartości przeznaczona jest metoda *transform*.

In [ ]:
new_num_attributes = imputer.transform(num_attributes)

In [ ]:
import pandas as pd

new_num_attributes = pd.DataFrame(new_num_attributes, columns=data.columns)

In [ ]:
new_num_attributes

In [ ]:
new_num_attributes.isnull().any(axis=0)

Z uwagi na różnice między interfejsami bibliotek *pandas* i *Scikit-learn* warto zwrócić uwagę na typowe aspekty interfejsu aktualnie używanego narzędzia. Wykorzystywane są dwie niezależne metody: *fit* oraz *transform*. Wywołanie metody fit oznacza dopasowanie do aktualnie przekazanej ramki danych i wyznaczenie na jej podstawie wartości do uzupełnienia. Analogicznie wygląda sytuacja dla przekazanego atrybutu. Wywołanie metody *transform* spowoduje faktyczne uzupełnienie brakujących wartości i zwrócenie utworzonego w ten sposób nowego obiektu będącego pełną ramką danych. Warto więc mieć na uwadze, że próba wywołania metody *transform* po wywołaniu metody *fit* na innym zbiorze danych może kompletnie mijać się z celem.

## Zadania

1. Pobrać i wczytać zbiór danych danych dostępny pod adresem: https://archive.ics.uci.edu/dataset/10/automobile

2. Wartości wybrakowane w zbiorze oznaczone są symbolem "?". W celu zamiany na wartość *None* - dopasowaną do języka Python - można wykorzystać metodę *replace* wywoływaną na obiekcie ramki *pandas*. Utworzyć głęboką kopię powstałej w ten sposób ramki danych (funkcja [deepcopy](https://docs.python.org/3/library/copy.html)).

3. Zastosować poznane metody uzupełniania wartości wybrakowanych dopasowanych do typów danych w atrybutach za pomocą biblioteki *pandas*. W przypadku gdy do danego typu pasuje więcej niż jedna strategia (np. do typu ciągłego numerycznego: mediana i średnia arytmetyczna), utworzyć dwie wersje kolumn (np. col1\_mean i col1\_median), gdzie w każdej będą wartości uzupełnione inną strategią.

4. Na podstawie utworzonej kopii ramki z punktu 2, powtórzyć kroki z punktu 3, ale przy użyciu biblioteki *Scikit-learn*.

